## EDA para el dataset iris

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# %matplotlib notebook
%matplotlib inline

Importamos los datos del dataset iris.data en un panda y mostramos su información básica mediante la instrucción info()

In [ ]:
iris=pd.read_csv('iris.data')
iris.info()

Imprimimos los primeros y los últimos datos del dataset

In [ ]:
print(iris.head())
print(iris.tail())

Comenzamos con un análisis exploratorio de datos no visual, de cada una de las características individualmente. Para ello, utilizamos la función describe(), que muestra las estadísticas básicas, excluyendo los valores nulos (en este dataset no hay ninguno)

In [ ]:
iris.describe()

Para cada uno de los atributos, analizamos visualmente sus valores

In [ ]:
iris.plot.hist(subplots=True, bins=20);

In [ ]:
iris.plot.box()

Como este dataset es válido para un problema de clasificación, es mejor realizar este análisis teniendo en cuenta la clase de cada ejemplo

In [ ]:
agrupado=iris.groupby('Clase')
agrupado.describe()

In [ ]:
agrupado.groups.keys()

In [ ]:
plt.figure()
graf1=plt.subplot(2,2,1)
graf2=plt.subplot(2,2,2)
graf3=plt.subplot(2,2,3)
graf4=plt.subplot(2,2,4)
for name, group in agrupado:
    graf1.plot(group['Largo sepalo'], marker='o', linestyle='')
for name, group in agrupado:
    graf2.plot(group['Ancho sepalo'], marker='o', linestyle='')
for name, group in agrupado:
    graf3.plot(group['Largo petalo'], marker='o', linestyle='')
for name, group in agrupado:
    graf4.plot(group['Ancho petalo'], marker='o', linestyle='')
plt.legend(agrupado.groups.keys(),bbox_to_anchor=(-0.4, 1), loc='lower left', borderaxespad=0.)


Cuidado con la información que mostramos. ¿Cuántas dimensiones estamos mostrando? ¿Cuáles son estas dimensiones? Volvemos a mostrar los datos eliminando el número de dato

In [ ]:
plt.figure()
graf1=plt.subplot(4,1,1)
graf2=plt.subplot(4,1,2)
graf3=plt.subplot(4,1,3)
graf4=plt.subplot(4,1,4)
agrupado=iris.groupby('Clase')
for name, group in agrupado:
    graf1.plot(group['Largo sepalo'],np.zeros_like(group['Largo sepalo']), marker='o', linestyle='')
for name, group in agrupado:
    graf2.plot(group['Ancho sepalo'],np.zeros_like(group['Ancho sepalo']), marker='o', linestyle='')
for name, group in agrupado:
    graf3.plot(group['Largo petalo'],np.zeros_like(group['Largo petalo']), marker='o', linestyle='')
for name, group in agrupado:
    graf4.plot(group['Ancho petalo'],np.zeros_like(group['Ancho petalo']), marker='o', linestyle='')

Existen dos nuevos tipos de gráficas que nos ayudan a ver la separabilidad de las clases en cada uno de los atributos: las curvas de Andrew y las coordenadas paralelas.
* Curvas de Andrew: permiten mostrar datos multivariable, utilizando el valor de cada atributos como coeficientes de Fourier
* Coordenadas paralelas: permiten mostrar datos multivariable, para intentar mostrar clusters

Para utilizar cualquiera de estas gráficas es necesario importarlas, desde pandas.tools.plotting

In [ ]:
from pandas.plotting import andrews_curves
from pandas.plotting import parallel_coordinates
plt.figure()
andrews_curves(iris,'Clase')
plt.figure()
parallel_coordinates(iris,'Clase')

Comprobamos que 'Largo petalo' y 'Ancho sépalo' son los dos atributos que mejor separan a las 3 clases. También vemos que la clase Iris-setosa es completamente identificable teniendo en cuenta cada uno de estos atributos individualmente.

Viasualizamos los datos cuando tenemos en cuenta estos dos atributos al mismo tiempo

In [ ]:
colores = np.where(iris.Clase == 'Iris-virginica', 'r',np.where(iris.Clase=='Iris-setosa','g','b'))
iris.plot.scatter(x='Largo petalo', y='Ancho petalo',c=colores)


### Clasificación de los datos

Una vez que hemos analizado el dataset y hemos obtenido algunas conclusiones, vamos a comprobar si estas son ciertas empíricamente.
Como hemos dicho anteriormente, el dataset consta de 150 datos, 50 de cada una de las clases. Vamos a dividir el dataset en dos conjuntos:
* Conjunto de entrenamiento: con 120 datos aleatorios del dataset
* Conjunto de test: con los 30 datos restantes

El objetivo es clasificar los 30 datos del conjunto de test basándonos en los datos de entrenamiento. Como sabemos la clase real de estos ejemplos, podemos comprobar la tasa de acierto de nuestra clasificación

Empezamos creando nuestros dos conjuntos

In [ ]:
#Aleatorizamos el dataset
np.random.seed(0)
aleat=np.random.permutation(np.arange(150))

#Separamos en dos
entrenamiento=iris.iloc[aleat[0:120]]
test=iris.iloc[aleat[120:150]]

Para cada dato de test buscamos cuál es el dato de entrenamiento más cercano, y decimos que pertenece a la misma clase que este dato. Comprobamos si la clase es la correcta, y almacenamos el resultado. 
Este estudio lo hacemos teniendo en cuenta cada uno de los 4 atributos por separado

In [ ]:
#Largo sepalo
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,0]
    dTestC=test.iloc[i,[0,4]]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,0]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos

In [ ]:
#Ancho sepalo
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,1]
    dTestC=test.iloc[i,[1,4]]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,1]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos

In [ ]:
#Largo petalo
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,2]
    dTestC=test.iloc[i,[2,4]]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,2]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos

In [ ]:
#Ancho petalo
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,3]
    dTestC=test.iloc[i,[3,4]]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,3]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos

In [ ]:
#Largo y ancho petalo
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,[2,3]]
    dTestC=test.iloc[i,[2,3,4]]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,[2,3]]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos

In [ ]:
#Todos atributos
aciertos=pd.DataFrame(np.zeros((3,3)),columns=['Iris-setosa','Iris-versicolor','Iris-virginica'],index=['Iris-setosa','Iris-versicolor','Iris-virginica'])
for i in np.arange(30):
    dTest=test.iloc[i,:-1]
    dTestC=test.iloc[i,:]
    distancias=np.zeros((120,1))
    for j in np.arange(120):
        dTrain=entrenamiento.iloc[j,:-1]
        distancias[j]=((dTrain-dTest)**2).sum()
    posi=distancias.argmin()
    aciertos.loc[dTestC['Clase'],entrenamiento.iloc[posi,4]]+=1
aciertos